In [17]:
from config import *


In [18]:
print(f"{year}-{month:02}")

2020-07


In [19]:
import pandas as pd
import sys
sys.path.append('../../py')
import db
import weighted
import inspect
import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif']=['SimHei'] #用来正常显示中文标签
plt.rcParams['axes.unicode_minus']=False #用来正常显示负号
%matplotlib inline

import seaborn as sns
import scipy.stats as stats
import numpy as np
import math


In [20]:
from matplotlib.font_manager import _rebuild

_rebuild() #reload一下

In [ ]:
#from map_wrapper import *
#from mpl_toolkits.basemap import Basemap

In [21]:
conn=db.get_conn()

In [47]:

cities = """
('北京','上海','深圳','杭州','广州','南京','苏州','成都','东莞','西安','武汉','天津','长沙',
'宁波','福州','大连','重庆','青岛','济南','合肥','长春','昆明','郑州','沈阳','哈尔滨','厦门')
"""
data_original=pd.read_sql(sql=f"select * from jobs where yearmonth={year}{month:02} and monthly_salary>0 and monthly_salary<80000 and city in {cities}", con=conn)


In [48]:
error_job_ids=['104660258','104142922','108434795','101357291','106253516','110368302','111391233','108665401','109277048'
                  ,'73857191','108584955','102824950','102824949','111391233','110884556']
data=data_original[~data_original.job_id.isin(error_job_ids)]

In [49]:
del data['publish_date']
del data['published_on_weekend']
del data['title']
del data['job_description']
del data['job_id']

In [50]:
#Common Functions
def get_sub_stats_by_col(data, col):
    categories=data[col].unique()
    salary_mean=[]
    salary_95_min=[]
    salary_95_max=[]
    salary_median=[]

    count=[]
    
    categorys_out=[]
    for category in categories:
        #print(feature)
        idata=data[data[col]==category]
        headcount=idata.headcount.sum()
        values = idata.monthly_salary.values
        weights = idata.headcount.values
        #print(str(headcount))
        if headcount==0:
            continue
        
        salary_mean.append(np.average(values, weights=weights))
        

        q = weighted.weighted_quantile(values,[0.025,0.5,0.975],weights)
        salary_95_min.append(q[0])
        salary_median.append(q[1])
        salary_95_max.append(q[2])
        count.append(idata.headcount.sum())
        categorys_out.append(category)
    sub_data=pd.DataFrame()
    sub_data[col]=[c for c in categorys_out]
    sub_data['平均工资']=salary_mean
    sub_data['最低工资']=salary_95_min
    sub_data['中位数']=salary_median
    sub_data['最高工资']=salary_95_max
    sub_data['招聘人数']=count
    sub_data['百分比']=count/np.sum(count)
    sub_data=sub_data.sort_values(by='平均工资', ascending=False)

    return sub_data

data_format={"百分比":"{:.2%}","平均工资":"{:.0f}","中位数":"{:.0f}","最低工资":"{:.0f}","最高工资":"{:.0f}"}

translate_dict={}


# 程序员工资

In [51]:
data_city=get_sub_stats_by_col(data,'city')
data_city['排名']=list(range(1,data_city.shape[0]+1))
data_city=data_city[['排名', 'city', '平均工资', '最低工资', '中位数', '最高工资', '招聘人数', '百分比']]
data_city.style.format(data_format)

,排名,city,平均工资,最低工资,中位数,最高工资,招聘人数,百分比
3,1,北京,19333,7000,17500,45000,22447,8.60%
0,2,上海,17423,7000,15000,40000,47031,18.02%
2,3,深圳,17048,7000,15000,37500,40315,15.44%
6,4,杭州,16421,6500,14583,37500,14978,5.74%
8,5,广州,14135,5250,12500,30000,29635,11.35%
1,6,南京,13990,5250,12500,30000,14648,5.61%
14,7,东莞,13153,5250,12500,25000,3142,1.20%
4,8,成都,12857,4500,12000,30000,13650,5.23%
11,9,苏州,12805,5250,12500,27121,8020,3.07%
22,10,厦门,12792,3750,12500,25000,1853,0.71%


In [52]:
def describe(data_city, career):
    for index, row in data_city.iterrows():
        print(f"{year}年{month}月{row['city']}招收{career}{row['招聘人数']}人。2019年{month}月{row['city']}{career}平均工资{row['平均工资']:.0f}元，工资中位数{row['中位数']:.0f}元，其中95%的人的工资介于{row['最低工资']:.0f}元到{row['最高工资']:.0f}元。\r\n")

In [53]:
career='软件工程师'
for index, row in data_city.iterrows():
    print(f"{year}年{month}月{row['city']}招收{career}{row['招聘人数']}人。2019年{month}月{row['city']}{career}平均工资{row['平均工资']:.0f}元，工资中位数{row['中位数']:.0f}元，其中95%的人的工资介于{row['最低工资']:.0f}元到{row['最高工资']:.0f}元。\r\n")

2020年7月北京招收软件工程师22447人。2019年7月北京软件工程师平均工资19333元，工资中位数17500元，其中95%的人的工资介于7000元到45000元。

2020年7月上海招收软件工程师47031人。2019年7月上海软件工程师平均工资17423元，工资中位数15000元，其中95%的人的工资介于7000元到40000元。

2020年7月深圳招收软件工程师40315人。2019年7月深圳软件工程师平均工资17048元，工资中位数15000元，其中95%的人的工资介于7000元到37500元。

2020年7月杭州招收软件工程师14978人。2019年7月杭州软件工程师平均工资16421元，工资中位数14583元，其中95%的人的工资介于6500元到37500元。

2020年7月广州招收软件工程师29635人。2019年7月广州软件工程师平均工资14135元，工资中位数12500元，其中95%的人的工资介于5250元到30000元。

2020年7月南京招收软件工程师14648人。2019年7月南京软件工程师平均工资13990元，工资中位数12500元，其中95%的人的工资介于5250元到30000元。

2020年7月东莞招收软件工程师3142人。2019年7月东莞软件工程师平均工资13153元，工资中位数12500元，其中95%的人的工资介于5250元到25000元。

2020年7月成都招收软件工程师13650人。2019年7月成都软件工程师平均工资12857元，工资中位数12000元，其中95%的人的工资介于4500元到30000元。

2020年7月苏州招收软件工程师8020人。2019年7月苏州软件工程师平均工资12805元，工资中位数12500元，其中95%的人的工资介于5250元到27121元。

2020年7月厦门招收软件工程师1853人。2019年7月厦门软件工程师平均工资12792元，工资中位数12500元，其中95%的人的工资介于3750元到25000元。

2020年7月西安招收软件工程师8077人。2019年7月西安软件工程师平均工资12416元，工资中位数11500元，其中95%的人的工资介于5102元到27500元。

2020年7月长沙招收软件工程师6240人。2019年7月长沙软件工程师平均工资12073元，

In [54]:
describe(data_city,'软件工程师')

2020年7月北京招收软件工程师22447人。2019年7月北京软件工程师平均工资19333元，工资中位数17500元，其中95%的人的工资介于7000元到45000元。

2020年7月上海招收软件工程师47031人。2019年7月上海软件工程师平均工资17423元，工资中位数15000元，其中95%的人的工资介于7000元到40000元。

2020年7月深圳招收软件工程师40315人。2019年7月深圳软件工程师平均工资17048元，工资中位数15000元，其中95%的人的工资介于7000元到37500元。

2020年7月杭州招收软件工程师14978人。2019年7月杭州软件工程师平均工资16421元，工资中位数14583元，其中95%的人的工资介于6500元到37500元。

2020年7月广州招收软件工程师29635人。2019年7月广州软件工程师平均工资14135元，工资中位数12500元，其中95%的人的工资介于5250元到30000元。

2020年7月南京招收软件工程师14648人。2019年7月南京软件工程师平均工资13990元，工资中位数12500元，其中95%的人的工资介于5250元到30000元。

2020年7月东莞招收软件工程师3142人。2019年7月东莞软件工程师平均工资13153元，工资中位数12500元，其中95%的人的工资介于5250元到25000元。

2020年7月成都招收软件工程师13650人。2019年7月成都软件工程师平均工资12857元，工资中位数12000元，其中95%的人的工资介于4500元到30000元。

2020年7月苏州招收软件工程师8020人。2019年7月苏州软件工程师平均工资12805元，工资中位数12500元，其中95%的人的工资介于5250元到27121元。

2020年7月厦门招收软件工程师1853人。2019年7月厦门软件工程师平均工资12792元，工资中位数12500元，其中95%的人的工资介于3750元到25000元。

2020年7月西安招收软件工程师8077人。2019年7月西安软件工程师平均工资12416元，工资中位数11500元，其中95%的人的工资介于5102元到27500元。

2020年7月长沙招收软件工程师6240人。2019年7月长沙软件工程师平均工资12073元，

In [38]:
#draw_city_map(data_city,2000,f'{year}年{month}月中国大陆主要城市程序员工资')

# 区块链工程师

In [55]:
data_blockchain=get_sub_stats_by_col(data[(data['career']=='区块链开发')],'city')

data_blockchain.style.format(data_format)

,city,平均工资,最低工资,中位数,最高工资,招聘人数,百分比
10,大连,30000,30000,30000,30000,1,0.14%
14,厦门,21595,16500,20000,30000,58,8.01%
2,杭州,20877,12125,21000,47875,54,7.46%
5,北京,20290,9000,17083,40000,50,6.91%
3,深圳,20131,11500,20500,29750,90,12.43%
18,合肥,19167,17500,19167,22500,3,0.41%
8,南京,19119,11525,17500,25000,21,2.90%
4,上海,18840,9000,17500,39250,106,14.64%
11,宁波,18611,17500,18611,20833,3,0.41%
1,广州,18338,10680,15000,30100,176,24.31%


In [56]:
describe(data_blockchain,'区块链工程师')

2020年7月大连招收区块链工程师1人。2019年7月大连区块链工程师平均工资30000元，工资中位数30000元，其中95%的人的工资介于30000元到30000元。

2020年7月厦门招收区块链工程师58人。2019年7月厦门区块链工程师平均工资21595元，工资中位数20000元，其中95%的人的工资介于16500元到30000元。

2020年7月杭州招收区块链工程师54人。2019年7月杭州区块链工程师平均工资20877元，工资中位数21000元，其中95%的人的工资介于12125元到47875元。

2020年7月北京招收区块链工程师50人。2019年7月北京区块链工程师平均工资20290元，工资中位数17083元，其中95%的人的工资介于9000元到40000元。

2020年7月深圳招收区块链工程师90人。2019年7月深圳区块链工程师平均工资20131元，工资中位数20500元，其中95%的人的工资介于11500元到29750元。

2020年7月合肥招收区块链工程师3人。2019年7月合肥区块链工程师平均工资19167元，工资中位数19167元，其中95%的人的工资介于17500元到22500元。

2020年7月南京招收区块链工程师21人。2019年7月南京区块链工程师平均工资19119元，工资中位数17500元，其中95%的人的工资介于11525元到25000元。

2020年7月上海招收区块链工程师106人。2019年7月上海区块链工程师平均工资18840元，工资中位数17500元，其中95%的人的工资介于9000元到39250元。

2020年7月宁波招收区块链工程师3人。2019年7月宁波区块链工程师平均工资18611元，工资中位数18611元，其中95%的人的工资介于17500元到20833元。

2020年7月广州招收区块链工程师176人。2019年7月广州区块链工程师平均工资18338元，工资中位数15000元，其中95%的人的工资介于10680元到30100元。

2020年7月东莞招收区块链工程师5人。2019年7月东莞区块链工程师平均工资18000元，工资中位数18000元，其中95%的人的工资介于18000元到18000元。

2020年7月长沙招收区块链工程师26人。2019年7月长沙区块链工程师平均工资15577元，

In [57]:
#draw_city_map(data_blockchain[data_blockchain['招聘人数']>10],10,f'{year}年{month}月中国大陆主要城市区块链工程师工资')

# 机器学习工程师

In [58]:
data_city_algorithm=get_sub_stats_by_col(data[data.career=='机器学习'],'city')
data_city_algorithm.style.format(data_format)

,city,平均工资,最低工资,中位数,最高工资,招聘人数,百分比
1,北京,27854,10206,27500,50786,126,16.34%
6,深圳,25660,7417,21667,54917,130,16.86%
0,上海,24381,7950,18750,62500,98,12.71%
3,杭州,17990,7400,17500,30000,48,6.23%
21,福州,17875,11500,17875,20000,4,0.52%
5,南京,17800,9000,18125,27500,25,3.24%
18,厦门,17500,17500,17500,17500,2,0.26%
9,广州,16187,1750,16333,29812,29,3.76%
16,大连,16125,15000,15286,20000,16,2.08%
7,合肥,15833,15000,15833,17500,3,0.39%


In [59]:
describe(data_city_algorithm,'机器学习')

2020年7月北京招收机器学习126人。2019年7月北京机器学习平均工资27854元，工资中位数27500元，其中95%的人的工资介于10206元到50786元。

2020年7月深圳招收机器学习130人。2019年7月深圳机器学习平均工资25660元，工资中位数21667元，其中95%的人的工资介于7417元到54917元。

2020年7月上海招收机器学习98人。2019年7月上海机器学习平均工资24381元，工资中位数18750元，其中95%的人的工资介于7950元到62500元。

2020年7月杭州招收机器学习48人。2019年7月杭州机器学习平均工资17990元，工资中位数17500元，其中95%的人的工资介于7400元到30000元。

2020年7月福州招收机器学习4人。2019年7月福州机器学习平均工资17875元，工资中位数17875元，其中95%的人的工资介于11500元到20000元。

2020年7月南京招收机器学习25人。2019年7月南京机器学习平均工资17800元，工资中位数18125元，其中95%的人的工资介于9000元到27500元。

2020年7月厦门招收机器学习2人。2019年7月厦门机器学习平均工资17500元，工资中位数17500元，其中95%的人的工资介于17500元到17500元。

2020年7月广州招收机器学习29人。2019年7月广州机器学习平均工资16187元，工资中位数16333元，其中95%的人的工资介于1750元到29812元。

2020年7月大连招收机器学习16人。2019年7月大连机器学习平均工资16125元，工资中位数15286元，其中95%的人的工资介于15000元到20000元。

2020年7月合肥招收机器学习3人。2019年7月合肥机器学习平均工资15833元，工资中位数15833元，其中95%的人的工资介于15000元到17500元。

2020年7月西安招收机器学习26人。2019年7月西安机器学习平均工资14798元，工资中位数13846元，其中95%的人的工资介于2725元到22500元。

2020年7月沈阳招收机器学习6人。2019年7月沈阳机器学习平均工资14167元，工资中位数10000元，其中95%的人的工资介于5000元到37500元。

2020年7月长沙招收机器

In [60]:
#draw_city_map(data_city_algorithm,100,f'{year}年{month}月中国大陆主要城市机器学习工资')

# 系统架构师

In [61]:
data_city_architect=get_sub_stats_by_col(data[data.career=='架构师'],'city')
data_city_architect.style.format(data_format)

,city,平均工资,最低工资,中位数,最高工资,招聘人数,百分比
4,北京,31661,12500,30000,60000,891,14.62%
3,深圳,31060,12500,30000,56000,984,16.15%
1,上海,30975,12500,30000,55000,1215,19.94%
5,杭州,28956,14062,29166,45000,470,7.71%
2,广州,25443,11992,25000,45833,738,12.11%
7,南京,25122,11228,25000,39964,302,4.96%
10,成都,24202,9200,21250,50500,212,3.48%
21,厦门,23485,10000,25000,42156,33,0.54%
12,苏州,21687,2579,17917,75000,96,1.58%
17,东莞,21065,11558,22500,35000,27,0.44%


In [62]:
describe(data_city_architect,'架构师')

2020年7月北京招收架构师891人。2019年7月北京架构师平均工资31661元，工资中位数30000元，其中95%的人的工资介于12500元到60000元。

2020年7月深圳招收架构师984人。2019年7月深圳架构师平均工资31060元，工资中位数30000元，其中95%的人的工资介于12500元到56000元。

2020年7月上海招收架构师1215人。2019年7月上海架构师平均工资30975元，工资中位数30000元，其中95%的人的工资介于12500元到55000元。

2020年7月杭州招收架构师470人。2019年7月杭州架构师平均工资28956元，工资中位数29166元，其中95%的人的工资介于14062元到45000元。

2020年7月广州招收架构师738人。2019年7月广州架构师平均工资25443元，工资中位数25000元，其中95%的人的工资介于11992元到45833元。

2020年7月南京招收架构师302人。2019年7月南京架构师平均工资25122元，工资中位数25000元，其中95%的人的工资介于11228元到39964元。

2020年7月成都招收架构师212人。2019年7月成都架构师平均工资24202元，工资中位数21250元，其中95%的人的工资介于9200元到50500元。

2020年7月厦门招收架构师33人。2019年7月厦门架构师平均工资23485元，工资中位数25000元，其中95%的人的工资介于10000元到42156元。

2020年7月苏州招收架构师96人。2019年7月苏州架构师平均工资21687元，工资中位数17917元，其中95%的人的工资介于2579元到75000元。

2020年7月东莞招收架构师27人。2019年7月东莞架构师平均工资21065元，工资中位数22500元，其中95%的人的工资介于11558元到35000元。

2020年7月大连招收架构师84人。2019年7月大连架构师平均工资20806元，工资中位数18500元，其中95%的人的工资介于11500元到45571元。

2020年7月天津招收架构师27人。2019年7月天津架构师平均工资20750元，工资中位数22500元，其中95%的人的工资介于8500元到32358元。

2020年7月武汉招收架构师264人。2019

In [63]:
#draw_city_map(data_city_architect,40,f'{year}年{month}月中国大陆主要城市【架构师】工资')

# 生物信息工程师

In [64]:
data_bio=get_sub_stats_by_col(data[data.career=='生物信息'],'city')
data_bio.style.format(data_format)

,city,平均工资,最低工资,中位数,最高工资,招聘人数,百分比
4,杭州,20833,20833,20833,20833,1,5.00%
2,宁波,16000,16000,16000,16000,1,5.00%
5,广州,13583,11500,13583,14000,6,30.00%
3,长沙,12500,12500,12500,12500,3,15.00%
1,上海,11800,8000,11800,17500,5,25.00%
0,苏州,10750,5500,12500,12500,4,20.00%


In [65]:
describe(data_bio,'生物信息工程师')

2020年7月杭州招收生物信息工程师1人。2019年7月杭州生物信息工程师平均工资20833元，工资中位数20833元，其中95%的人的工资介于20833元到20833元。

2020年7月宁波招收生物信息工程师1人。2019年7月宁波生物信息工程师平均工资16000元，工资中位数16000元，其中95%的人的工资介于16000元到16000元。

2020年7月广州招收生物信息工程师6人。2019年7月广州生物信息工程师平均工资13583元，工资中位数13583元，其中95%的人的工资介于11500元到14000元。

2020年7月长沙招收生物信息工程师3人。2019年7月长沙生物信息工程师平均工资12500元，工资中位数12500元，其中95%的人的工资介于12500元到12500元。

2020年7月上海招收生物信息工程师5人。2019年7月上海生物信息工程师平均工资11800元，工资中位数11800元，其中95%的人的工资介于8000元到17500元。

2020年7月苏州招收生物信息工程师4人。2019年7月苏州生物信息工程师平均工资10750元，工资中位数12500元，其中95%的人的工资介于5500元到12500元。



In [67]:
#draw_city_map(data_bio,2,f'{year}年{month}月中国大陆主要城市【生物信息工程师】工资')

# 爬虫工程师

In [68]:
data_spider=get_sub_stats_by_col(data[data.career=='爬虫开发工程师'],'city')
data_spider=data_spider[data_spider['招聘人数']>10]
data_spider.style.format(data_format)

,city,平均工资,最低工资,中位数,最高工资,招聘人数,百分比
10,北京,16969,7254,15000,37375,81,5.82%
0,上海,16223,7000,15000,30000,198,14.23%
4,杭州,15883,6183,15000,37500,64,4.60%
2,深圳,14290,7000,12500,29609,295,21.21%
8,南京,14203,5250,14000,24000,74,5.32%
3,广州,12982,5475,12500,21883,137,9.85%
19,合肥,12224,3750,9000,22500,16,1.15%
9,武汉,12028,5571,11500,17500,136,9.78%
23,东莞,11562,9500,11500,15000,16,1.15%
6,宁波,11180,10000,11035,13333,12,0.86%


In [69]:
describe(data_spider, '爬虫开发工程师')

2020年7月北京招收爬虫开发工程师81人。2019年7月北京爬虫开发工程师平均工资16969元，工资中位数15000元，其中95%的人的工资介于7254元到37375元。

2020年7月上海招收爬虫开发工程师198人。2019年7月上海爬虫开发工程师平均工资16223元，工资中位数15000元，其中95%的人的工资介于7000元到30000元。

2020年7月杭州招收爬虫开发工程师64人。2019年7月杭州爬虫开发工程师平均工资15883元，工资中位数15000元，其中95%的人的工资介于6183元到37500元。

2020年7月深圳招收爬虫开发工程师295人。2019年7月深圳爬虫开发工程师平均工资14290元，工资中位数12500元，其中95%的人的工资介于7000元到29609元。

2020年7月南京招收爬虫开发工程师74人。2019年7月南京爬虫开发工程师平均工资14203元，工资中位数14000元，其中95%的人的工资介于5250元到24000元。

2020年7月广州招收爬虫开发工程师137人。2019年7月广州爬虫开发工程师平均工资12982元，工资中位数12500元，其中95%的人的工资介于5475元到21883元。

2020年7月合肥招收爬虫开发工程师16人。2019年7月合肥爬虫开发工程师平均工资12224元，工资中位数9000元，其中95%的人的工资介于3750元到22500元。

2020年7月武汉招收爬虫开发工程师136人。2019年7月武汉爬虫开发工程师平均工资12028元，工资中位数11500元，其中95%的人的工资介于5571元到17500元。

2020年7月东莞招收爬虫开发工程师16人。2019年7月东莞爬虫开发工程师平均工资11562元，工资中位数11500元，其中95%的人的工资介于9500元到15000元。

2020年7月宁波招收爬虫开发工程师12人。2019年7月宁波爬虫开发工程师平均工资11180元，工资中位数11035元，其中95%的人的工资介于10000元到13333元。

2020年7月长春招收爬虫开发工程师13人。2019年7月长春爬虫开发工程师平均工资10577元，工资中位数10577元，其中95%的人的工资介于10000元到12500元。

2020年7月重庆招收爬虫开发工程师22人。2019年

In [70]:
#draw_city_map(data_spider,5,f'{year}年{month}月中国大陆主要城市【爬虫工程师】工资')

# Java程序员

In [71]:
data_java=get_sub_stats_by_col(data[(data.pl_java==1)],'city')
data_java.style.format(data_format)

,city,平均工资,最低工资,中位数,最高工资,招聘人数,百分比
2,北京,18873,7000,16000,45000,9171,9.70%
1,深圳,17478,7500,15500,37500,12658,13.39%
4,上海,17444,7000,16000,37500,16479,17.43%
12,杭州,16949,7000,15000,40000,5548,5.87%
6,广州,14736,6500,13000,30000,11609,12.28%
0,南京,14007,5250,12500,30000,6136,6.49%
13,东莞,13030,5500,12500,22500,674,0.71%
19,厦门,12963,3750,12500,22855,719,0.76%
8,大连,12562,5000,10000,32906,2714,2.87%
11,苏州,12447,5500,11500,25000,2220,2.35%


In [72]:
describe(data_java, 'Java程序员')

2020年7月北京招收Java程序员9171人。2019年7月北京Java程序员平均工资18873元，工资中位数16000元，其中95%的人的工资介于7000元到45000元。

2020年7月深圳招收Java程序员12658人。2019年7月深圳Java程序员平均工资17478元，工资中位数15500元，其中95%的人的工资介于7500元到37500元。

2020年7月上海招收Java程序员16479人。2019年7月上海Java程序员平均工资17444元，工资中位数16000元，其中95%的人的工资介于7000元到37500元。

2020年7月杭州招收Java程序员5548人。2019年7月杭州Java程序员平均工资16949元，工资中位数15000元，其中95%的人的工资介于7000元到40000元。

2020年7月广州招收Java程序员11609人。2019年7月广州Java程序员平均工资14736元，工资中位数13000元，其中95%的人的工资介于6500元到30000元。

2020年7月南京招收Java程序员6136人。2019年7月南京Java程序员平均工资14007元，工资中位数12500元，其中95%的人的工资介于5250元到30000元。

2020年7月东莞招收Java程序员674人。2019年7月东莞Java程序员平均工资13030元，工资中位数12500元，其中95%的人的工资介于5500元到22500元。

2020年7月厦门招收Java程序员719人。2019年7月厦门Java程序员平均工资12963元，工资中位数12500元，其中95%的人的工资介于3750元到22855元。

2020年7月大连招收Java程序员2714人。2019年7月大连Java程序员平均工资12562元，工资中位数10000元，其中95%的人的工资介于5000元到32906元。

2020年7月苏州招收Java程序员2220人。2019年7月苏州Java程序员平均工资12447元，工资中位数11500元，其中95%的人的工资介于5500元到25000元。

2020年7月西安招收Java程序员2728人。2019年7月西安Java程序员平均工资12295元，工资中位数11500元，其中95%的人的工资介于5250元到27500元。

2020年7月成都

In [73]:
#draw_city_map(data_java, 500, f'{year}年{month}月中国大陆主要城市【Java工程师】工资')

In [74]:
history_data = pd.read_sql(sql='select * from CityStats', con=conn)

ProgrammingError: (pyodbc.ProgrammingError) ('42S02', "[42S02] [Microsoft][SQL Server Native Client 11.0][SQL Server]Invalid object name 'CityStats'. (208) (SQLExecDirectW)")
[SQL: select * from CityStats]
(Background on this error at: http://sqlalche.me/e/f405)

In [ ]:
#plt.plot("Month","Salary",data=data_hangzhou, label='杭州')
#plt.plot("Month","Salary",data=history_data[history_data.City=='南京'], label='南京')
plt.figure(figsize=(12,6))
cities = set(history_data.City)
cities = ['北京','上海','广州','深圳','杭州']
for city in cities:
    plt.plot("Month","Salary",data=history_data[history_data.City==city], label=city)
    
    #plt.show()
plt.title('主要城市工资变化')
plt.legend()
plt.show()

In [ ]:
#plt.plot("Month","Salary",data=data_hangzhou, label='杭州')
#plt.plot("Month","Salary",data=history_data[history_data.City=='南京'], label='南京')
plt.figure(figsize=(15,8))
cities = set(history_data.City)
#cities = ['北京','上海','广州','深圳','杭州']
for city in cities:
    values=history_data[history_data.City==city]['Salary'].values
    if abs(values[-1]-values[-2])>1000:
        plt.plot("Month","Salary",data=history_data[history_data.City==city], label=city)
    
    #plt.show()
plt.title('主要城市工资变化')
plt.legend( loc='upper left')
plt.show()

In [ ]:
values=history_data[history_data.City==city]['Salary'].values